<a href="https://colab.research.google.com/github/HCA97/CLIP-based-NSFW-Detector/blob/cleanup/CLIP_based_NSFW_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/HCA97/CLIP-based-NSFW-Detector
%cd CLIP-based-NSFW-Detector
!pip install -r requirements.txt

Cloning into 'CLIP-based-NSFW-Detector'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 135 (delta 19), reused 20 (delta 14), pack-reused 107
Receiving objects: 100% (135/135), 25.96 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/CLIP-based-NSFW-Detector
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.4/162.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import torch.nn.functional as F
import torch
from PIL import Image

from src.inference import build_inference_model
from src.utils import remove_transparency

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [17]:
@torch.no_grad()
def predict_single_image_path(img_path, head, backbone, pre_processing):

    img = Image.open(img_path)
    img = remove_transparency(img)
    x = pre_processing(img).to(device)
    x = x.reshape(1, 3, 224, 224)
    p = backbone(x) 
    p = F.normalize(p)
    c = head(p)

    print(f'{img_path} is NSFW score ({c[0][0]})')

In [14]:
!wget https://www.damenmode-quelle.de/fotky421/fotos/_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg -O test.jpg

--2023-05-28 19:38:38--  https://www.damenmode-quelle.de/fotky421/fotos/_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg
Resolving www.damenmode-quelle.de (www.damenmode-quelle.de)... 81.0.206.116, 2001:1528:114::661c
Connecting to www.damenmode-quelle.de (www.damenmode-quelle.de)|81.0.206.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175773 (172K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>] 171.65K   733KB/s    in 0.2s    

2023-05-28 19:38:39 (733 KB/s) - ‘test.jpg’ saved [175773/175773]



# ViT-B-32

In [18]:
head, backbone, pre_procesing = build_inference_model('./models/clip_ViT-B-32_openai_binary_nsfw_head.pth', 
                                                      'ViT-B-32', 'openai', device)

In [19]:
predict_single_image_path('./test.jpg', head, backbone, pre_procesing)

./test.jpg is NSFW score (0.9998005032539368)


# ViT-L-14

In [20]:
head, backbone, pre_procesing = build_inference_model('./models/clip_ViT-L-14_openai_binary_nsfw_head.pth', 
                                                      'ViT-L-14', 'openai', device)

In [21]:
predict_single_image_path('./test.jpg', head, backbone, pre_procesing)

./test.jpg is NSFW score (0.9988253712654114)


# ViT-H-14

In [22]:
head, backbone, pre_procesing = build_inference_model('./models/clip_ViT-H-14_laion2b_s32b_b79k_binary_nsfw_head.pth', 
                                                      'ViT-H-14', 'laion2b_s32b_b79k', device)

In [23]:
predict_single_image_path('./test.jpg', head, backbone, pre_procesing)

./test.jpg is NSFW score (0.9861899614334106)
